In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-25 16:50:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Đang phân giải raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Kết nối tới raw.githubusercontent.com (raw.githubusercontent.com)[185.199.109.133]:443… đã kết nối.
Đã gửi yêu cầu HTTP, đang đợi câu trả lời… 200 OK
Kích thước: 1115394 (1.1M) [text/plain]
Đang ghi vào: “input.txt.8”

input.txt.8         100%[===================>]   1.06M  3.89MB/s    trong 0.3s 

2024-06-25 16:50:18 (3.89 MB/s) — đã lưu “input.txt.8” [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [10]:
import torch # we use PyTorch: https://pytorch.org
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
data = torch.tensor(encode(text), dtype=torch.long)

batch_size = 4
block_size = 32
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
block_size = 8
train_data[:block_size+1]
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]

def get_batch(split):
    #get the AI small batch of data of input x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:block_size + 1] for i in ix])
    return x,y

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets):
        logits = self.token_embedding_table(idx)
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits,targets) #loss function
        return logits,loss
    def generate(self,idx,max_new_token):
        for _ in range(max_new_tokens):
            #get the predictions
            logits,loss = self(idx)
            #focus on the last timestep
            logits = logits[:-1:]
            #apply softmax to get probabilities
            probs = F.softmax(logits,dim = 1) #(B,C)
            #sample from the distribution
            idx_next = torch.multinomial(probs,num_samples = 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx,idx_next),dim = 1)
        return idx
m = BigramLanguageModel(vocab_size)
xb, yb = get_batch('train')
logits, loss = m(xb, yb)
optimizer = torch.optim.AdamW(m.parameter)

RuntimeError: shape '[32]' is invalid for input of size 0

In [11]:
if torch.backends.mps.is_available():
    print("Metal Performance Shaders (MPS) backend is available.")
elif torch.backends.cuda.is_available():
    print("Cuda backend is available.")

Metal Performance Shaders (MPS) backend is available.


In [12]:
from transformers import GPT2LMHeadModel

/Users/dangkhoa/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
